<a href="https://colab.research.google.com/github/PremGorecki/NeuralNetwork/blob/main/02_basics/06_gradient_descent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Przygotowanie środowiska

In [ ]:
# Przygotowanie środowiska do pracy z Tensorflow 2.0.
# Jeśli otrzymasz błąd podczas instalacji Tensorflow uruchom tę komórkę raz jeszcze.

!pip uninstall -y tensorflow
!pip install -q tensorflow==2.0.0

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.graph_objects as go
tf.__version__

'2.0.0'

In [ ]:
function = lambda w: w ** 2 - 4 * w #nasza funkcja straty
derivative = lambda w: 2 * w - 4  #nasza pochodna


w = np.arange(-10, 14, 0.1)  # dane wejściowe
loss = function(w)   # funkcja straty
w_0 = -8  # punkt startowy

tangent_line = lambda w: derivative(w_0) * (w - w_0) + function(w_0) # styczna do wykresu funkcji w punkcie w_0
tangent_line_values = tangent_line(w)

In [ ]:
df = pd.DataFrame({'w': w, 'loss': loss,'tangent_line': tangent_line_values})
df.head()

In [ ]:
fig = go.Figure(data=[go.Scatter(x=df['w'], y=df['loss'], name='function'),
                      go.Scatter(x=df['w'][:100], y=df['tangent_line'][:100], name='tangent line'),
                      go.Scatter(x=[w_0], y=[function(w_0)], marker_size=10, name='point')],
                layout=go.Layout(width=800, title='Loss Function'))
fig.show()

In [ ]:
#algorytm stochastycznego gradientu
max_iters = 10000 
iters = 0
w_0 = -1
previous_step_size = 1
learning_rate = 0.01
precision = 0.000001
derivative = lambda w: 2 * w - 4
points = []

while previous_step_size > precision and iters < max_iters:
    w_prev = w_0
    w_0 = w_0 - learning_rate * derivative(w_prev)
    previous_step_size = abs(w_0 - w_prev)
    iters += 1
    points.append(w_0)
    print(f'Iteracja # {iters}: obecny punkt: {w_0}')

print(f'Minimum lokalne w punkcie: {w_0}')

In [ ]:
#Robimy sobie taką funkcję
def gradient_descent(derivative_func, learning_rate=0.01, max_iters=10000,
                     precision=0.000001, w_0=-8, verbose=True):
    iters = 0
    previous_step_size = 1
    points = []

    while previous_step_size > precision and iters < max_iters:
        w_prev = w_0
        w_0 = w_0 - learning_rate * derivative_func(w_prev)
        previous_step_size = abs(w_0 - w_prev)
        iters += 1
        points.append(w_0)
        if verbose:
            print(f'Iteracja # {iters}: obecny punkt: {w_0}')

    
    print(f'Minimum lokalne w punkcie: {w_0}')
    return points

points = gradient_descent(derivative)

In [ ]:
points = pd.DataFrame({'point': points})
# points = points.reset_index()
points['value'] = function(points['point'])
points.head()

In [ ]:
fig = go.Figure(data=[go.Scatter(x=df['w'], y=df['loss'], name='function'),
                      go.Scatter(x=points['point'], y=points['value'], marker_size=5, name='point', mode='markers')],
                layout=go.Layout(width=1000, title='Loss Function'))
fig.show()

In [ ]:
#Funkcja która testuje learing rate
def test_lr(func, derivative_func, learning_rate, w_0=-8):
    points = gradient_descent(derivative_func, learning_rate=learning_rate, w_0=w_0, verbose=False)
    points = pd.DataFrame({'point': points})
    points = points.reset_index()
    points['value'] = func(points['point'])

    fig = go.Figure(data=[go.Scatter(x=df['w'], y=df['loss'], name='function'),
                      go.Scatter(x=points['point'], y=points['value'], marker_size=5, name='point', mode='markers+lines')],
                layout=go.Layout(width=1000, title=f'Loss Function learning_rate:{learning_rate}'))
    fig.show()

test_lr(function, derivative, 0.01)

In [ ]:
test_lr(function, derivative, 0.1)

In [ ]:
test_lr(function, derivative, 0.055)

In [ ]:
test_lr(function, derivative, 0.001)

In [ ]:
#Przykład z inną funkcją
function_2 = lambda w: (w + 8) * (w - 5) * (w - 10) * (w + 5) * (w + 5) * (w - 2)

#bibliotek symap liczy pochodną z funkcji
from sympy import Symbol, lambdify

w = Symbol('w')
f = (w + 8) * (w - 5) * (w - 10) * (w + 5) * (w + 5) * (w - 2)
f_diff = f.diff(w)
derivative_2 = lambdify(w, f_diff)
del w

In [ ]:
w = np.arange(-9, 10, 0.1)
loss = function_2(w)
w_0 = -4

tangent_line = lambda w: derivative_2(w_0) * (w - w_0) + function_2(w_0)
tangent_line_values = tangent_line(w)

df = pd.DataFrame({'w': w, 'loss': loss,'tangent_line': tangent_line_values})
df.head()

fig = go.Figure(data=[go.Scatter(x=df['w'], y=df['loss'], name='function_2'),
                      go.Scatter(x=df['w'][20:80], y=df['tangent_line'][20:80], name='tangent line'),
                      go.Scatter(x=[w_0], y=[function_2(w_0)], marker_size=10, name='point')],
                layout=go.Layout(width=800, title='Loss Function'))
fig.show()

In [ ]:
#To się wywali
gradient_descent(derivative_2, learning_rate=0.01, w_0=-4)

In [ ]:
#To wejdzie w minimum lokalne
gradient_descent(derivative_2, learning_rate=0.0001, w_0=-4)

In [ ]:
test_lr(function_2, derivative_2, learning_rate=0.0001, w_0=-4)

In [ ]:
#startujemy od 3ki ale też dojedziemy do tego samego minimum lokalnego
gradient_descent(derivative_2, learning_rate=0.0001, w_0=3)

In [ ]:
test_lr(function_2, derivative_2, learning_rate=0.0001, w_0=3)

In [ ]:
#zaczynamy od 4-y
gradient_descent(derivative_2, learning_rate=0.0001, w_0=4)

In [ ]:
#tu jest za duży wskaźnik uczenia
test_lr(function_2, derivative_2, learning_rate=0.0001, w_0=4)